In [ ]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.4 MB/s eta 0:00:00


In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Побудова Байєсівської мережі
model = BayesianNetwork([
    ('Asia', 'Tuberculosis'),
    ('Animals', 'Infection'),
    ('Tuberculosis', 'X-ray'),
    ('Infection', 'Disease'),
    ('Disease', 'X-ray')
])

# Визначення умовних ймовірностей вершин
cpd_asia = TabularCPD(variable='Asia', variable_card=2, values=[[0.99], [0.01]])
cpd_animals = TabularCPD(variable='Animals', variable_card=2, values=[[0.95], [0.05]])
cpd_tuberculosis = TabularCPD(variable='Tuberculosis', variable_card=2,
                              values=[[0.95, 0.2], [0.05, 0.8]],
                              evidence=['Asia'], evidence_card=[2])
cpd_infection = TabularCPD(variable='Infection', variable_card=2,
                           values=[[0.9, 0.3], [0.1, 0.7]],
                           evidence=['Animals'], evidence_card=[2])
cpd_disease = TabularCPD(variable='Disease', variable_card=2,
                         values=[[0.7, 0.2], [0.3, 0.8]],
                         evidence=['Infection'], evidence_card=[2])
cpd_xray = TabularCPD(variable='X-ray', variable_card=2,
                      values=[[0.9, 0.5, 0.7, 0.1], [0.1, 0.5, 0.3, 0.9]],
                      evidence=['Tuberculosis', 'Disease'], evidence_card=[2, 2])

# Додавання умовних ймовірностей у модель
model.add_cpds(cpd_asia, cpd_animals, cpd_tuberculosis, cpd_infection, cpd_disease, cpd_xray)

# Перевірка моделі на коректність
assert model.check_model(), "Модель побудована некоректно!"

# Використання алгоритму виключення змінних для обчислення ймовірностей
inference = VariableElimination(model)

# Обчислення ймовірностей станів для вершини 'X-ray'
result = inference.query(variables=['X-ray'], evidence={'Asia': 1, 'Animals': 1})
print(result)

+----------+--------------+
| X-ray    |   phi(X-ray) |
+==========+==============+
| X-ray(0) |       0.3760 |
+----------+--------------+
| X-ray(1) |       0.6240 |
+----------+--------------+


In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Створення структури Байєсівської мережі
model = BayesianNetwork([
    ('Animal', 'Environment'),
    ('Animal', 'HasShell'),
    ('Animal', 'BearsYoungAs'),
    ('Animal', 'Class'),
    ('Class', 'Warmblooded'),
    ('Class', 'BodyCovering')
])

# Таблиця ймовірностей для вузла Animal
cpd_animal = TabularCPD(
    variable='Animal', variable_card=5,
    values=[[0.2], [0.2], [0.2], [0.2], [0.2]],
    state_names={'Animal': ['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle']}
)

# Таблиця умовних ймовірностей для вузла Environment
cpd_environment = TabularCPD(
    variable='Environment', variable_card=3,
    values=[
        [0, 0, 0, 0.5, 0],  # Air
        [1, 0.5, 0, 0.5, 0.5],  # Land
        [0, 0.5, 1, 0, 0.5]  # Water
    ],
    evidence=['Animal'], evidence_card=[5],
    state_names={'Environment': ['air', 'land', 'water'],
                 'Animal': ['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle']}
)

# Таблиця умовних ймовірностей для вузла HasShell
cpd_has_shell = TabularCPD(
    variable='HasShell', variable_card=2,
    values=[
        [0, 0, 0, 0, 1],  # True
        [1, 1, 1, 1, 0]  # False
    ],
    evidence=['Animal'], evidence_card=[5],
    state_names={'HasShell': ['true', 'false'],
                 'Animal': ['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle']}
)

# Таблиця умовних ймовірностей для вузла BearsYoungAs
cpd_bears_young_as = TabularCPD(
    variable='BearsYoungAs', variable_card=2,
    values=[
        [1, 0, 0, 0, 0],  # Live
        [0, 1, 1, 1, 1]  # Eggs
    ],
    evidence=['Animal'], evidence_card=[5],
    state_names={'BearsYoungAs': ['live', 'eggs'],
                 'Animal': ['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle']}
)

# Таблиця умовних ймовірностей для вузла Class
cpd_class = TabularCPD(
    variable='Class', variable_card=3,
    values=[
        [0, 1, 0, 1, 0],  # Bird
        [1, 0, 1, 0, 0],  # Mammal
        [0, 0, 0, 0, 1]  # Reptile
    ],
    evidence=['Animal'], evidence_card=[5],
    state_names={'Class': ['bird', 'mammal', 'reptile'],
                 'Animal': ['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle']}
)

# Таблиця умовних ймовірностей для вузла Warmblooded
cpd_warmblooded = TabularCPD(
    variable='Warmblooded', variable_card=2,
    values=[
        [1, 1, 0],  # True
        [0, 0, 1]  # False
    ],
    evidence=['Class'], evidence_card=[3],
    state_names={'Warmblooded': ['true', 'false'],
                 'Class': ['bird', 'mammal', 'reptile']}
)

# Таблиця умовних ймовірностей для вузла BodyCovering
cpd_body_covering = TabularCPD(
    variable='BodyCovering', variable_card=3,
    values=[
        [0, 1, 0],  # Fur
        [1, 0, 0],  # Feathers
        [0, 0, 1]  # Scales
    ],
    evidence=['Class'], evidence_card=[3],
    state_names={'BodyCovering': ['fur', 'feathers', 'scales'],
                 'Class': ['bird', 'mammal', 'reptile']}
)

# Додавання умовних ймовірностей до моделі
model.add_cpds(cpd_animal, cpd_environment, cpd_has_shell, cpd_bears_young_as,
               cpd_class, cpd_warmblooded, cpd_body_covering)

# Перевірка правильності моделі
assert model.check_model()

# Виконання інференції
inference = VariableElimination(model)

# Обчислення ймовірності для прикладу
result = inference.query(variables=['Environment'], evidence={'Animal': 'Penguin'})
print(result)

# Обчислення спільної ймовірності
joint_prob = inference.query(variables=['Animal', 'Environment', 'HasShell', 'BearsYoungAs', 'Class', 'Warmblooded', 'BodyCovering'])
print(joint_prob)

+--------------------+--------------------+
| Environment        |   phi(Environment) |
+====================+====================+
| Environment(air)   |             0.0000 |
+--------------------+--------------------+
| Environment(land)  |             0.5000 |
+--------------------+--------------------+
| Environment(water) |             0.5000 |
+--------------------+--------------------+
+------------------+--------------------+-----------------+--------------------+----------------+--------------------+------------------------+--------------------------------------------------------------------------------+
| Animal           | Environment        | HasShell        | BearsYoungAs       | Class          | Warmblooded        | BodyCovering           |   phi(Animal,Environment,HasShell,BearsYoungAs,Class,Warmblooded,BodyCovering) |
+==================+====================+=================+====================+================+====================+========================+===========

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Створення структури Байєсівської мережі
model = BayesianNetwork([
    ('VisitAsia', 'Tuberculosis'),
    ('Smoking', 'LungCancer'),
    ('Smoking', 'Bronchitis'),
    ('Tuberculosis', 'Tb_or_Ca'),
    ('LungCancer', 'Tb_or_Ca'),
    ('Tb_or_Ca', 'XRay'),
    ('Tb_or_Ca', 'Dyspnea'),
    ('Bronchitis', 'Dyspnea')
])

# Таблиця ймовірностей для вузла VisitAsia
cpd_visit_asia = TabularCPD(
    variable='VisitAsia', variable_card=2,
    values=[[0.01], [0.99]],
    state_names={'VisitAsia': ['visit', 'no_visit']}
)

# Таблиця ймовірностей для вузла Smoking
cpd_smoking = TabularCPD(
    variable='Smoking', variable_card=2,
    values=[[0.5], [0.5]],
    state_names={'Smoking': ['smoking', 'no_smoking']}
)

# Таблиця умовних ймовірностей для вузла Tuberculosis
cpd_tuberculosis = TabularCPD(
    variable='Tuberculosis', variable_card=2,
    values=[[0.05, 0.01], [0.95, 0.99]],
    evidence=['VisitAsia'], evidence_card=[2],
    state_names={'Tuberculosis': ['present', 'absent'], 'VisitAsia': ['visit', 'no_visit']}
)

# Таблиця умовних ймовірностей для вузла LungCancer
cpd_lung_cancer = TabularCPD(
    variable='LungCancer', variable_card=2,
    values=[[0.1, 0.01], [0.9, 0.99]],
    evidence=['Smoking'], evidence_card=[2],
    state_names={'LungCancer': ['present', 'absent'], 'Smoking': ['smoking', 'no_smoking']}
)

# Таблиця умовних ймовірностей для вузла Tb_or_Ca
cpd_tb_or_ca = TabularCPD(
    variable='Tb_or_Ca', variable_card=2,
    values=[
        [1, 1, 1, 0],  # True
        [0, 0, 0, 1]  # False
    ],
    evidence=['Tuberculosis', 'LungCancer'], evidence_card=[2, 2],
    state_names={'Tb_or_Ca': ['true', 'false'],
                 'Tuberculosis': ['present', 'absent'],
                 'LungCancer': ['present', 'absent']}
)

# Таблиця умовних ймовірностей для вузла XRay
cpd_xray = TabularCPD(
    variable='XRay', variable_card=2,
    values=[[0.98, 0.05], [0.02, 0.95]],
    evidence=['Tb_or_Ca'], evidence_card=[2],
    state_names={'XRay': ['abnormal', 'normal'], 'Tb_or_Ca': ['true', 'false']}
)

# Таблиця умовних ймовірностей для вузла Bronchitis
cpd_bronchitis = TabularCPD(
    variable='Bronchitis', variable_card=2,
    values=[[0.6, 0.3], [0.4, 0.7]],
    evidence=['Smoking'], evidence_card=[2],
    state_names={'Bronchitis': ['present', 'absent'], 'Smoking': ['smoking', 'no_smoking']}
)

# Таблиця умовних ймовірностей для вузла Dyspnea
cpd_dyspnea = TabularCPD(
    variable='Dyspnea', variable_card=2,
    values=[
        [0.9, 0.7, 0.8, 0.1],  # True
        [0.1, 0.3, 0.2, 0.9]  # False
    ],
    evidence=['Tb_or_Ca', 'Bronchitis'], evidence_card=[2, 2],
    state_names={'Dyspnea': ['true', 'false'],
                 'Tb_or_Ca': ['true', 'false'],
                 'Bronchitis': ['present', 'absent']}
)

# Додавання умовних ймовірностей до моделі
model.add_cpds(cpd_visit_asia, cpd_smoking, cpd_tuberculosis, cpd_lung_cancer,
               cpd_tb_or_ca, cpd_xray, cpd_bronchitis, cpd_dyspnea)

# Перевірка правильності моделі
assert model.check_model()

# Виконання інференції
inference = VariableElimination(model)

# Обчислення ймовірності для прикладу
result = inference.query(variables=['Dyspnea'], evidence={'Smoking': 'smoking', 'VisitAsia': 'visit'})
print("Ймовірність Dyspnea за умовами куріння і візиту до Азії:", result)

# Обчислення спільної ймовірності
joint_prob = inference.query(variables=['VisitAsia', 'Smoking', 'Tuberculosis', 'LungCancer', 'Tb_or_Ca', 'XRay', 'Bronchitis', 'Dyspnea'])
print("Спільна ймовірність для всієї мережі Asia:")
print(joint_prob)

Ймовірність Dyspnea за умовами куріння і візиту до Азії: +----------------+----------------+
| Dyspnea        |   phi(Dyspnea) |
+================+================+
| Dyspnea(true)  |         0.5635 |
+----------------+----------------+
| Dyspnea(false) |         0.4365 |
+----------------+----------------+
Спільна ймовірність для всієї мережі Asia:
+---------------------+---------------------+-----------------------+---------------------+-----------------+----------------+---------------------+----------------+-----------------------------------------------------------------------------------+
| VisitAsia           | Smoking             | Tuberculosis          | LungCancer          | Tb_or_Ca        | XRay           | Bronchitis          | Dyspnea        |   phi(VisitAsia,Smoking,Tuberculosis,LungCancer,Tb_or_Ca,XRay,Bronchitis,Dyspnea) |
+=====================+=====================+=======================+=====================+=================+================+=====================+